In [1]:
#%%
import datetime
import pandas as pd
import numpy as np
import psycopg2
from utils.portfolio import *
from utils.base import *
from utils.report import *
from utils.config import *
from utils.instance import *
import os
import warnings
warnings.filterwarnings('ignore')



In [2]:
options, market = fetch_real_time_last_snapshot()
options = extract_call_options_features(options)
for i in range(len(options)):
    try:
      options['interest_rate_ask_price_1'][i]>1
    except:
        options['interest_rate_ask_price_1'][i] = np.nan

In [3]:
options.head(10)

,id_option,isin_option,symbol_option,name,open_price,close_price,last_trade_option,count,volume,value,...,min_allowed_price_base,in_the_money_percentage,ask_price_1_w_o_c,bid_price_1_w_o_c,leverage_ask_price_1,leverage_bid_price_1,leverage_last_trade,interest_rate_last_trade,interest_rate_ask_price_1,interest_rate_bid_price_1
0,20037653197111739,IRO9AGAH0121,ضترو4000,اختيارخ ص آگاه-7000-02/04/14,0.0,7200.0,7200.0,0,0.0,0.0,...,NaN,NaN,7000.0,3009.0,NaN,NaN,NaN,NaN,NaN,NaN
1,8116142932215303,IRO9AGAH0141,ضترو4002,اختيارخ ص آگاه-8000-02/04/14,0.0,6108.0,6050.0,0,0.0,0.0,...,NaN,NaN,6800.0,5501.0,NaN,NaN,NaN,NaN,NaN,NaN
2,12833810923774304,IRO9AGAH0161,ضترو4004,اختيارخ ص آگاه-10000-02/04/14,4750.0,4599.0,4680.0,9,120.0,551824000.0,...,NaN,NaN,4687.0,4402.0,NaN,NaN,NaN,NaN,NaN,NaN
3,54179059222245165,IRO9AGAH0181,ضترو4006,اختيارخ ص آگاه-12000-02/04/14,3100.0,2899.0,3098.0,5,33.0,95668000.0,...,NaN,NaN,3099.0,1701.0,NaN,NaN,NaN,NaN,NaN,NaN
4,9181823518983931,IRO9AGAH0191,ضترو4007,اختيارخ ص آگاه-13000-02/04/14,2300.0,2566.0,2719.0,25,250.0,641463000.0,...,NaN,NaN,2719.0,2301.0,NaN,NaN,NaN,NaN,NaN,NaN
5,54936847147016069,IRO9AGAH0211,ضترو4009,اختيارخ ص آگاه-15000-02/04/14,900.0,929.0,950.0,4,350.0,325000000.0,...,NaN,NaN,1300.0,952.0,NaN,NaN,NaN,NaN,NaN,NaN
6,6656725773350069,IRO9AGAH0221,ضترو4010,اختيارخ ص آگاه-16000-02/04/14,610.0,610.0,610.0,3,40.0,24400000.0,...,NaN,NaN,890.0,530.0,NaN,NaN,NaN,NaN,NaN,NaN
7,816070469779688,IRO9AHRM6261,ضهرم7000,اختيارخ اهرم-6500-1402/07/23,24200.0,24292.0,24600.0,5,707.0,17174710000.0,...,25440.0,0.779586,25390.0,23101.0,1.162678,1.275255,1.198780,0.876832,1.732739,0.047447
8,29567585335146943,IRO9AHRM6291,ضهرم7003,اختيارخ اهرم-8000-1402/07/23,0.0,21076.0,21500.0,0,0.0,0.0,...,25440.0,0.728722,22890.0,13002.0,1.289664,2.265779,1.371628,0.002771,0.518401,-0.797694
9,51752770340789810,IRO9AHRM6301,ضهرم7004,اختيارخ اهرم-9000-1402/07/23,0.0,20650.0,20800.0,0,0.0,0.0,...,25440.0,0.694812,22099.0,18983.0,1.335825,1.551897,1.417788,0.080623,0.535562,-0.287055


In [4]:
options.columns


Index(['id_option', 'isin_option', 'symbol_option', 'name', 'open_price',
       'close_price', 'last_trade_option', 'count', 'volume', 'value',
       'low_price', 'high_price', 'yesterday_price', 'table_id', 'group_id',
       'max_allowed_price', 'min_allowed_price_option', 'last_ret',
       'close_ret', 'number_of_shares', 'market_cap', 'bid_price_4',
       'bid_vol_4', 'ask_price_4', 'ask_vol_4', 'bid_price_3', 'bid_vol_3',
       'ask_price_3', 'ask_vol_3', 'bid_price_5', 'bid_vol_5', 'ask_price_5',
       'ask_vol_5', 'bid_price_1', 'bid_vol_1', 'ask_price_1', 'ask_vol_1',
       'bid_price_2', 'bid_vol_2', 'ask_price_2', 'ask_vol_2', 'ask_value_1',
       'ask_value_2', 'ask_value_3', 'ask_value_4', 'ask_value_5',
       'bid_value_1', 'bid_value_2', 'bid_value_3', 'bid_value_4',
       'bid_value_5', 'time', 'date', 'jalali_date', 'base_symbol',
       'jalali_due_date', 'days_until_due', 'miladi_due_date',
       'due_date_price', 'last_trade_base', 'symbol_base', 'id_base'

In [5]:
options['spread_pct'] = 100* (options['ask_price_1'] / options['bid_price_1']  - 1)

In [6]:
options[(options['value']>200e7 )& (options['spread_pct'] > 10) & \
        (options['last_trade_option'] /options['ask_price_1'] > 0.95 )&\
        (options['interest_rate_ask_price_1'] > 0.7)& (options['in_the_money_percentage'] > 0.2)]

,id_option,isin_option,symbol_option,name,open_price,close_price,last_trade_option,count,volume,value,...,in_the_money_percentage,ask_price_1_w_o_c,bid_price_1_w_o_c,leverage_ask_price_1,leverage_bid_price_1,leverage_last_trade,interest_rate_last_trade,interest_rate_ask_price_1,interest_rate_bid_price_1,spread_pct
326,52625011044902880,IROABDYZ2031,ضدی203,اختيارخ دي-600-14020217,1115.0,1226.0,1250.0,43,4138.0,5072446000.0,...,0.664242,1252.0,1110.0,1.428788,1.608253,1.429600,24896.057793,28092.871126,-0.999981,12.560679
353,50226428962234019,IROAPKOD2091,ضخپارس300,اختيارخ خپارس-1200-14020321,1290.0,1284.0,1391.0,35,1893.0,2431114000.0,...,0.451052,1400.0,1262.0,1.563039,1.730389,1.571531,46.154302,50.564123,10.696482,10.706733
357,55508390569984906,IROAPKOD2181,ضخپارس600,اختيارخ خپارس-1200-14020621,1314.0,1337.0,1600.0,40,2509.0,3355704000.0,...,0.451052,1600.0,1360.0,1.367659,1.605699,1.366250,6.257073,6.200936,1.822001,17.404955


In [7]:
pd.options.display.max_rows = 10
options_selected = options[options['value']>500e7].sort_values('value', ascending=False)

In [8]:
options_selected['IV'] = 0
options_selected['call_bs'] = 0

In [9]:
for i in range(options_selected.shape[0]):
    options_selected['IV'].iloc[i] = implied_volatility_call(options_selected['last_trade_option'].iloc[i],
                   options_selected['last_trade_base'].iloc[i],
                   options_selected['due_date_price'].iloc[i],
                   options_selected['days_until_due'].iloc[i]/365,
                   0.3,)
    if options_selected['IV'].iloc[i] > 0 :
        options_selected['call_bs'].iloc[i] = black_scholes_call(options_selected['last_trade_base'].iloc[i],
                                                            options_selected['due_date_price'].iloc[i],
                                                            options_selected['days_until_due'].iloc[i]/365,
                                                            0.4, options_selected['IV'].iloc[i])
    else:
                options_selected['call_bs'].iloc[i] = black_scholes_call(options_selected['last_trade_base'].iloc[i],
                                                            options_selected['due_date_price'].iloc[i],
                                                            options_selected['days_until_due'].iloc[i]/365,
                                                            0.6, 0.4)


    print('done!')

found on 21th iteration
difference is equal to -9.222386302099039e-05
done!
found on 12th iteration
difference is equal to -2.827608886946109e-05
done!
found on 8th iteration
difference is equal to -1.4379461163116503e-05
done!
found on 15th iteration
difference is equal to -3.6137372717348626e-05
done!
found on 5th iteration
difference is equal to -3.052876263609505e-05
done!
found on 29th iteration
difference is equal to -9.848214585872483e-05
done!
found on 27th iteration
difference is equal to -9.707667982183921e-05
done!
found on 7th iteration
difference is equal to -5.8248141840522294e-05
done!
done!
found on 15th iteration
difference is equal to -8.979166341305245e-05
done!
done!
found on 7th iteration
difference is equal to -3.1687436376159894e-05
done!
found on 33th iteration
difference is equal to -7.94166671767016e-05
done!
found on 16th iteration
difference is equal to -5.261966725811362e-05
done!
found on 25th iteration
difference is equal to -5.600852739462425e-05
done!
d

In [10]:
import plotly.express as px
options_selected = options_selected.reset_index(drop=True)
px.line(options_selected , x='symbol_option' , y='IV')

In [11]:
options_selected['diff'] = options_selected['last_trade_option'] - options_selected['call_bs']

In [12]:
px.line(options_selected , x='symbol_option' , y='diff')

In [13]:
pd.options.display.max_rows = 100

options_selected[options_selected['symbol_option']=='ضجار6000'].T

,25
id_option,52157458348940396
isin_option,IRO9BTEJ4551
symbol_option,ضجار6000
name,اختيارخ وتجارت-1500-1402/06/14
open_price,1431.0
close_price,1482.0
last_trade_option,1520.0
count,179
volume,11628.0
value,17237346000.0


In [14]:
black_scholes_call(2114,1500,135/365 , 0.75, 1.5)

1174.5177300541886

In [15]:
options_selected['ITM'] = options_selected['ITM'].astype(str)
px.scatter_3d(options_selected , x='days_until_due' , y='interest_rate_last_trade',z='in_the_money_percentage',
           hover_data=['symbol_option','diff'] ,color='ITM')

KeyError: 'ITM'

In [16]:
options_selected['ITM'] = 1

In [17]:
options_selected['ITM'][options_selected['due_date_price'] > options_selected['last_trade_option'] ] = 0

In [18]:
options_selected['interest_rate_last_trade'][options_selected['ITM']==1].mean()

2771.980627857295

In [19]:
options_selected['interest_rate_last_trade'][(options_selected['ITM']==0 )& (options_selected['interest_rate_last_trade']<np.inf)].mean()

3.046115164546379

In [20]:
class BlackScholes:
    """ 
    Class to calculate (European) call and put option prices through the Black-Scholes formula 
    without dividends
    
    :param S: Price of underlying stock
    :param K: Strike price
    :param T: Time till expiration (in years)
    :param r: Risk-free interest rate (0.05 indicates 5%)
    :param sigma: Volatility (standard deviation) of stock (0.15 indicates 15%)
    """
    @staticmethod
    def _d1(S, K, T, r, sigma):
        return (1 / (sigma * np.sqrt(T))) * (np.log(S/K) + (r + sigma**2 / 2) * T)
    
    def _d2(self, S, K, T, r, sigma):
        return self._d1(S, K, T, r, sigma) - sigma * np.sqrt(T)
    
    def call_price(self, S, K, T, r, sigma):
        """ Main method for calculating price of a call option """
        d1 = self._d1(S, K, T, r, sigma)
        d2 = self._d2(S, K, T, r, sigma)
        return norm.cdf(d1) * S - norm.cdf(d2) * K * np.exp(-r*T)
    
    def put_price(self, S, K, T, r, sigma):
        """ Main method for calculating price of a put option """
        d1 = self._d1(S, K, T, r, sigma)
        d2 = self._d2(S, K, T, r, sigma)
        return norm.cdf(-d2) * K * np.exp(-r*T) - norm.cdf(-d1) * S
    
    def call_in_the_money(self, S, K, T, r, sigma):
        """ 
        Calculate probability that call option will be in the money at
        maturity according to Black-Scholes.
        """
        d2 = self._d2(S, K, T, r, sigma)
        return norm.cdf(d2)
    
    def put_in_the_money(self, S, K, T, r, sigma):
        """ 
        Calculate probability that put option will be in the money at
        maturity according to Black-Scholes.
        """
        d2 = self._d2(S, K, T, r, sigma)
        return 1 - norm.cdf(d2)

In [21]:
def call_implied_volatility(price, S, K, T, r):
    """ Calculate implied volatility of a call option up to 2 decimals of precision. """
    sigma = 0.0001
    while sigma < 1:
        d1 = BlackScholes()._d1(S, K, T, r, sigma)
        d2 = BlackScholes()._d2(S, K, T, r, sigma)
        price_implied = S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)
        if price - price_implied < 0.0001:
            return sigma
        sigma += 0.0001
    return "Not Found"

def put_implied_volatility(price, S, K, T, r):
    """ Calculate implied volatility of a put option up to 2 decimals of precision. """
    sigma = 0.0001
    while sigma < 1:
        call = BlackScholes().call_price(S, K, T, r, sigma)
        price_implied = K * np.exp(-r*T) - S + call
        if price - price_implied < 0.0001:
            return sigma
        sigma += 0.0001
    return "Not Found"

In [22]:
class OptionStrategies:
    @staticmethod
    def short_straddle(S, K, T, r, sigma):
        call = BlackScholes().call_price(S, K, T, r, sigma)
        put = BlackScholes().put_price(S, K, T, r, sigma)
        return - put - call
    
    @staticmethod
    def long_straddle(S, K, T, r, sigma):
        call = BlackScholes().call_price(S, K, T, r, sigma)
        put = BlackScholes().put_price(S, K, T, r, sigma)
        return put + call
    
    @staticmethod
    def short_strangle(S, K1, K2, T, r, sigma):
        assert K1 < K2, f"Please make sure that K1 < K2. Now K1={K1}, K2={K2}"
        put = BlackScholes().put_price(S, K1, T, r, sigma)
        call = BlackScholes().call_price(S, K2, T, r, sigma)
        return - put - call
    
    @staticmethod
    def long_strangle(S, K1, K2, T, r, sigma):
        assert K1 < K2, f"Please make sure that K1 < K2. Now K1={K1}, K2={K2}"
        put = BlackScholes().put_price(S, K1, T, r, sigma)
        call = BlackScholes().call_price(S, K2, T, r, sigma)
        return put + call
    
    @staticmethod
    def short_put_butterfly(S, K1, K2, K3, T, r, sigma):
        assert K1 < K2 < K3, f"Please make sure that K1 < K2 < K3. Now K1={K1}, K2={K2}, K3={K3}"
        put1 = BlackScholes().put_price(S, K1, T, r, sigma)
        put2 = BlackScholes().put_price(S, K2, T, r, sigma)
        put3 = BlackScholes().put_price(S, K3, T, r, sigma)
        return - put1 + 2 * put2 - put3
    
    @staticmethod
    def long_call_butterfly(S, K1, K2, K3, T, r, sigma):
        assert K1 < K2 < K3, f"Please make sure that K1 < K2 < K3. Now K1={K1}, K2={K2}, K3={K3}"
        call1 = BlackScholes().call_price(S, K1, T, r, sigma)
        call2 = BlackScholes().call_price(S, K2, T, r, sigma)
        call3 = BlackScholes().call_price(S, K3, T, r, sigma)
        return call1 - 2 * call2 + call3
    
    @staticmethod
    def short_iron_condor(S, K1, K2, K3, K4, T, r, sigma):
        assert K1 < K2 < K3 < K4, f"Please make sure that K1 < K2 < K3 < K4. Now K1={K1}, K2={K2}, K3={K3}, K4={K4}"
        put1 = BlackScholes().put_price(S, K1, T, r, sigma)
        put2 = BlackScholes().put_price(S, K2, T, r, sigma)
        call1 = BlackScholes().call_price(S, K3, T, r, sigma)
        call2 = BlackScholes().call_price(S, K4, T, r, sigma)
        return put1 - put2 - call1 + call2
    
    @staticmethod
    def long_iron_condor(S, K1, K2, K3, K4, T, r, sigma):
        assert K1 < K2 < K3 < K4, f"Please make sure that K1 < K2 < K3 < K4. Now K1={K1}, K2={K2}, K3={K3}, K4={K4}"
        put1 = BlackScholes().put_price(S, K1, T, r, sigma)
        put2 = BlackScholes().put_price(S, K2, T, r, sigma)
        call1 = BlackScholes().call_price(S, K3, T, r, sigma)
        call2 = BlackScholes().call_price(S, K4, T, r, sigma)

        return - put1 + put2 + call1 - call2